In [3]:
import qminospy
from qminospy.me2 import ME_NLP
import cobra
import cobrame
import pickle

In [4]:
with open('./me_models/final/iJT964-ME.pickle', 'rb') as f:
    me = pickle.load(f)

In [5]:
me.solution

<Solution 0.10 at 0x7ff69bfc8ad0>

In [5]:
for r in me.reactions:
    lb,ub = r.bounds
    lb = lb.subs('mu',me.solution.x_dict["biomass_dilution"]) if hasattr(lb,'subs') else lb
    ub = ub.subs('mu',me.solution.x_dict["biomass_dilution"]) if hasattr(ub,'subs') else ub
    if me.solution.x_dict[r.id] < lb or me.solution.x_dict[r.id] > ub:
        print(r)
        print(lb,ub)
        print(me.solution.x_dict[r.id])
        print()

transcription_TU8J2_598_from_BSU25200-MONOMER
0.0 1000.0
-6.929970924267043e-43

translation_BSU08030
0.0 1000.0
-1.5315643006859103e-25

translation_BSU28470
0.0 1000.0
-1.559428457669845e-41

DM_RNA_BSU39390
0.0 1000.0
-7.144080468263394e-32

formation_BSU08030-MONOMER
0.0 1000.0
-1.5315643006859103e-25

formation_BSU28470-MONOMER
0.0 1000.0
-1.7886420370199704e-41

formation_CPLX000-44
0.0 1000.0
-5.77506678475192e-25

FACOAL161_ISO_FWD_BSU28560-MONOMER_mod_mg2
0 999999.0
-1.3287911217845254e-18

KAS5_FWD_CPLX000-44
0 999999.0
-1.3287911217845254e-18

DNA_replication
0.0195741620046771 0.0195741620046771
0.019574162004677126



In [21]:
import pandas
df = pandas.DataFrame.from_dict({"flux":me.solution.x_dict})
df = df[df.abs()>1e-18].dropna()
df.loc[df.abs().sort_values('flux').index]

,flux
FACOAL161_ISO_FWD_BSU28560-MONOMER_mod_mg2,-1.328791e-18
KAS5_FWD_CPLX000-44,-1.328791e-18
TAGURr_FWD_BSU12380-MONOMER,7.630429e-18
TAGURr_REV_BSU12380-MONOMER,7.630429e-18
DM_RNA_BSU27870,9.815412e-17
...,...
CYOO3_FWD_CPLX8J2-22,1.155276e+01
CYOR3m_FWD_CPLX8J2-42,1.155276e+01
ATPS4r_FWD_CPLX000-10,1.904623e+01
CYTK2_REV_BSU22890-MONOMER_mod_mg2,1.947901e+02


In [3]:
len(me.reactions),len(me.metabolites)

(6277, 4211)

In [4]:
bd = {
    'formation':[],
    'translation':[],
    'transcription':[],
    'charging':[],
    'metabolic':[],
    'dilution':[],
    'other':[]
    
}
for r in me.reactions:
    if 'formation' in r.id:
        bd['formation'].append(r.id)
    elif 'translation' in r.id:
        bd['translation'].append(r.id)
    elif 'transcription' in r.id:
        bd['transcription'].append(r.id)
    elif 'charging' in r.id:
        bd['charging'].append(r.id)
    elif 'FWD' in r.id or 'REV' in r.id:
        bd['metabolic'].append(r.id)
    elif 'dilution' in r.id:
        bd['dilution'].append(r.id)
    else:
        bd['other'].append(r.id)

In [5]:
for k,v in bd.items():
    print(k,len(v))

formation 867
translation 965
transcription 742
charging 306
metabolic 1783
dilution 1
other 1613


In [10]:
[g.id for g in me.metabolites if 'RNA_BSU' in g.id]

['RNA_BSU_rRNA_1',
 'RNA_BSU_tRNA_5',
 'RNA_BSU_tRNA_6',
 'RNA_BSU_rRNA_2',
 'RNA_BSU_rRNA_3',
 'RNA_BSU00090',
 'RNA_BSU00110',
 'RNA_BSU00120',
 'RNA_BSU00130',
 'RNA_BSU_tRNA_7',
 'RNA_BSU00140',
 'RNA_BSU00150',
 'RNA_BSU00180',
 'RNA_BSU_misc_RNA_2',
 'RNA_BSU_rRNA_4',
 'RNA_BSU_tRNA_8',
 'RNA_BSU_tRNA_9',
 'RNA_BSU_rRNA_5',
 'RNA_BSU_rRNA_26',
 'RNA_BSU00270',
 'RNA_BSU00280',
 'RNA_BSU00380',
 'RNA_BSU00410',
 'RNA_BSU00460',
 'RNA_BSU00500',
 'RNA_BSU00510',
 'RNA_BSU00550',
 'RNA_BSU_tRNA_10',
 'RNA_BSU_tRNA_11',
 'RNA_BSU00680',
 'RNA_BSU00730',
 'RNA_BSU00740',
 'RNA_BSU00750',
 'RNA_BSU00760',
 'RNA_BSU00770',
 'RNA_BSU00780',
 'RNA_BSU00790',
 'RNA_BSU00810',
 'RNA_BSU00820',
 'RNA_BSU_rRNA_6',
 'RNA_BSU_rRNA_7',
 'RNA_BSU_rRNA_8',
 'RNA_BSU_tRNA_12',
 'RNA_BSU_tRNA_13',
 'RNA_BSU_tRNA_14',
 'RNA_BSU_tRNA_15',
 'RNA_BSU_tRNA_16',
 'RNA_BSU_tRNA_17',
 'RNA_BSU_tRNA_18',
 'RNA_BSU_tRNA_19',
 'RNA_BSU_tRNA_20',
 'RNA_BSU_rRNA_9',
 'RNA_BSU_rRNA_10',
 'RNA_BSU_rRNA_11',
 'RNA_

In [13]:
import re
file = open('genes.txt','w')
for g in me.metabolites.query(re.compile('RNA_BSU')):
    gi = g.id[4:]
    if len(g.reactions) > 2:
        file.write('{}\n'.format(gi))
file.close()

In [6]:
def solve_me_model(me, max_mu=1., precision=1e-6, min_mu=0, using_soplex=True,
                  compiled_expressions=None, verbosity = 2, mu_fix = False,
                  growth_key='mu'):
    from qminospy.me1 import ME_NLP1
    ## If fixed growth rate, solve as LP
    if mu_fix:
        me_nlp = ME_NLP1(me)
        me_nlp.solvelp(mu_fix)
    else:
        ## 
        if using_soplex:
            from cobrame.solve.algorithms import binary_search
            binary_search(me, min_mu=min_mu, max_mu=max_mu, debug=True, mu_accuracy=precision,
                compiled_expressions=compiled_expressions)
        else:
            # The object containing solveME methods--composite that uses a ME model object 
            me_nlp = ME_NLP1(me, growth_key=growth_key)
            # Use bisection for now (until the NLP formulation is worked out)
            muopt, hs, xopt, cache = me_nlp.bisectmu(precision=precision, mumax=max_mu, verbosity=verbosity)
            try:
                me.solution.f = me.solution.x_dict['biomass_dilution']
            except:
                pass


In [7]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-2)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 29.358602 seconds
Finished substituting S,lb,ub in 1.003206 seconds
Finished makeME_LP in 0.144962 seconds
Getting MINOS parameters from ME_NLP...
 Allocating name1, name2
 Allocating pi, rc, xn
 Allocated pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
3FFF0000000000000000000000000000
 3FF0000000000000
3FFF0000000000000000000000000000
C083D6329F1C35CA5000000000000000
 C83D6329F1C35CA5
C083D6329F1C35CA5000000000000000
4083D6329F1C35CA5000000000000000
 483D6329F1C35CA5
4083D6329F1C35CA5000000000000000
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set option Print frequency to       100000
 Calling miopti to set option Scale option to            2
 Calling mi

In [52]:
solve_me_model(me, max_mu = 0.5, min_mu = .05, using_soplex=False, precision = 1e-2)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 133.019908 seconds
Finished substituting S,lb,ub in 3.686620 seconds
Finished makeME_LP in 0.710011 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 3.735425 seconds
Finished makeME_LP in 0.702127 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 3.614719 seconds
Finished makeME_LP in 0.716447 seconds
Getting MINOS parameters from ME_NLP...
3 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 3.652693 seconds
Finished makeME_LP in 0.721774 seconds
Getting MINOS parameters from ME_NLP...
4 0.09375 0.09375 0.125 0.09375 optimal
Finished substituting S,lb,ub in 4.117332 seconds
Finished makeME_LP in 0.723346 seconds
Getting MINOS parameters from ME_NLP...
5 0.09375 0.09375 0.109375 0.109375 1
Finished substituting S,lb,ub in 3.809529 seconds
Finished makeME_LP in 0.833264 seconds
Gettin